In [1]:
import numpy as np
import netwin as nw
import os
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
g = nx.gnp_random_graph(5, 0.5)
A = nx.adjacency_matrix(g)
D = np.diag([g.degree[i] for i in range(5)])
L = D - A
L = np.array(L)

In [3]:
def network_diffusion(p, t, theta):
    L, k = theta
    du = k * (np.matmul(-L, p))
    return du

def solve(f, p, t, theta):
    return odeint(f, p, t, args=(theta,))

def forward(f, u0, t, params): 
    p = u0[:-1]
    theta = params, u0[-1]
    #
    # print(u0)
    u = solve(f, p, t, theta) 

    return u.T 

In [4]:
    def setpriors(init_means):

        m0 = np.zeros_like(init_means)
        p0 = np.linalg.inv(np.diag(np.ones_like(m0) * 1e5))

        beta_mean0 = 1.0
        beta_var0  = 1000.0

        c0 = beta_var0 / beta_mean0
        s0 = beta_mean0**2 / beta_var0

        priors = m0, p0, c0, s0

        return priors

    def initialise(init_means, priors):
        if priors == None:
            priors = setpriors(init_means)
        
        m = init_means
        p = np.linalg.inv(np.diag(np.ones_like(m)))
        #c = np.array([priors[2]])
        #s = np.array([priors[3]])
        c = np.array([1e-8])
        s = np.array([50.0])
        params = m, p, c, s
        
        return params, priors

In [5]:
""" Scipt containing functions to implement variational inference using 
multivariate normal distribution and gamma distribution
"""

import numpy as np

def time_step(theta): 
    """Control time stepping interval proportional to the parameter magnitiude. 

    args: 
    theta:: float 
    """

    delta = theta * 1e-5
    if delta.any() < 0:
            delta = -delta
    if delta.any() < 1e-10:
            delta = 1e-10
    
    return delta

def central_difference(f, i, theta, delta, t):
    """ Calculate the derivate using a first order central difference approximation

    args:
        d : function 
            function on which to compute central difference 
        i : int 
            parameter index
    theta : np.array float 
            array of paramters for function, f
    delta : float 
            time-step for central difference
        t : np.array float
            time steps to evaluate at
    returns: 
        df : np.array, float 
             first order approximation, df, for time steps, t
    """
    dtheta = np.array(theta), np.array(theta)
    dtheta[0][i] += delta
    dtheta[1][i] -= delta
    f_1 = f(f=network_diffusion, u0=dtheta[0], t=t, params=L) 
    f_2 = f(f=network_diffusion, u0=dtheta[1], t=t, params=L)
    den = (2 * delta)
    df = (f_1 - f_2) / den
    return df

def Jacobian(f, theta, t):
    """Compute the Jacobian for globally defined function, f, with parameter set Theta 

    args:
    theta : np.array, float
            parameters of global function, f
        t : np.array, float 
            time steps to evaluate at

    returns: 
        J : np.array, float
            Jacobian vector/matrix evaluated at theta, t. 
    """
    J = None
    #f_n = len(f)
    p_n = len(theta)

    for i in range(p_n):
        delta = time_step(theta[i])
        if J is None:
                J = np.zeros([len(theta[:-1]) * len(t), len(theta)], dtype=np.float32)
        df = central_difference(f, i, theta, delta, t)
        J[:,i] = df.flatten()
    
    return J

def parameter_update(error, params, priors, J):
    """ Update forward model function parameters theta in accordance with the update equations above
    
    args: 
        error : array, float
                vector of the difference between observations and model prediction 
       params : tuple
                parameters values
       priors : tuple 
                priors values
            J : array, float 
                array of Jacbian values from the model given parameter updates (calculated above)
    returns: 
       params : tuple
                updated parameters values
    """
    m, p, s, c = params
    m0, p0, _, _ = priors

    p_new = s*c*np.dot(J.transpose(), J) + p0
    c_new = np.linalg.inv(p_new)
    print(error.shape)
    m_new = np.dot(c_new, (s * c * np.dot(J.transpose(), (error.flatten() +    np.dot(J, m))) + np.dot(p0, m0)))
    
    params[0][:], params[1][:] = m_new, p_new

    return params

def noise_update(error, data, params, priors, J):
    """ Update forward model function parameters phi in accordance with the update equations above
    
    args: 
        error : array, float
                vector of the difference between observations and model prediction 
       params : tuple
                parameters values
       priors : tuple 
                priors values
            J : array, float 
                array of Jacbian values from the model given parameter updates (calculated above)
    returns: 
       params : tuple
                updated parameters values
    """
    _, p, _, _ = params 
    _, _, s0, c0 = priors

    N = len(data)
    c = np.linalg.inv(p)
    c_new = N/2 + c0
    s_new = 1/(1/s0 + 1/2 * np.dot(error.flatten().transpose(), error.flatten()) + 1/2 * np.trace(np.dot(c, np.dot(J.transpose(), J))))
    
    params[2][:], params[3][:] = c_new, s_new

    return params

def error_update(y, f, theta, t):
    """Calculate difference between data and model with updated parameters

    args:
        y : array, float 
            vector of noisy data (observations)
    theta : array, float
            vector of parameter values for model, f
        t : array, float 
            vector of time steps at which to evaluate model 
    
    returns:
    error : array, float 
            vector of difference between noisy data and updated model
    """

    error = y - f(f=network_diffusion, u0=theta[0], t=t, params=L)
    
    return error

def fit(f, data, params, priors, t, n): 
    
    theta = np.zeros((n, len(params[0])))

    for i in range(n):
        theta[i,:] = params[0]

        error = error_update(data, f, params, t)

        J = Jacobian(f, params[0], t)
        print(J.shape)
        params = parameter_update(error, params, priors, J)
        params = noise_update(error, data, params, priors, J)
        print(params)
    return params, theta

In [26]:
p = np.zeros([5])
p[2] = 10.0
k = 1.0

u0 = np.append(p, k)
params = L

t = np.linspace(0,1,100)

p0 = np.ones([5])
k0 = 1

u_0 = np.append(p0, k0)

In [27]:
u = forward(f=network_diffusion,u0=u0,t=t,params=params)

In [28]:
params, priors = initialise(init_means=u_0, priors=None)
params[0]

array([1., 1., 1., 1., 1., 1.])

In [29]:
fi = fit(f=forward, data=u, params=params, priors=priors, t=t, n=500)

81094e-05, -1.46085690e-04,
         2.56081094e-05,  2.56081178e-05,  1.26136749e-03]]), array([2.501]), array([7.0437103e-07]))
(500, 6)
(5, 100)
(array([0.39536242, 0.74075424, 6.84521044, 0.74075506, 0.74075431,
       0.62176636]), array([[ 7.71658199e-05,  2.94555904e-05,  2.06305498e-05,
         2.94555971e-05,  2.94556005e-05,  6.92613666e-05],
       [ 2.94555904e-05,  9.25839165e-05,  2.94556038e-05,
         1.73340554e-05,  1.73340554e-05,  2.56081022e-05],
       [ 2.06305498e-05,  2.94556038e-05,  7.71658468e-05,
         2.94555971e-05,  2.94556005e-05, -1.46085619e-04],
       [ 2.94555971e-05,  1.73340554e-05,  2.94555971e-05,
         7.78157335e-05,  3.21022249e-05,  2.56080871e-05],
       [ 2.94556005e-05,  1.73340554e-05,  2.94556005e-05,
         3.21022249e-05,  7.78157335e-05,  2.56080921e-05],
       [ 6.92613666e-05,  2.56081022e-05, -1.46085619e-04,
         2.56080871e-05,  2.56080921e-05,  1.26136715e-03]]), array([2.501]), array([7.04371026e-07]))
(500, 

In [25]:
fi

((array([0.39536245, 0.74075389, 6.84521107, 0.74075485, 0.74075423,
         0.62176671]),
  array([[ 7.71658288e-05,  2.94555880e-05,  2.06305404e-05,
           2.94556014e-05,  2.94556014e-05,  6.92613956e-05],
         [ 2.94555880e-05,  9.25839258e-05,  2.94556048e-05,
           1.73340492e-05,  1.73340459e-05,  2.56081265e-05],
         [ 2.06305404e-05,  2.94556048e-05,  7.71658691e-05,
           2.94555947e-05,  2.94555980e-05, -1.46085745e-04],
         [ 2.94556014e-05,  1.73340492e-05,  2.94555947e-05,
           7.78157357e-05,  3.21022226e-05,  2.56081248e-05],
         [ 2.94556014e-05,  1.73340459e-05,  2.94555980e-05,
           3.21022226e-05,  7.78157424e-05,  2.56081232e-05],
         [ 6.92613956e-05,  2.56081265e-05, -1.46085745e-04,
           2.56081248e-05,  2.56081232e-05,  1.26136794e-03]]),
  array([2.501]),
  array([7.04371038e-07])),
 array([[0.39536266, 0.74075422, 6.84521053, 0.74075462, 0.74075448,
         0.6217666 ],
        [0.39536285, 0.74075422